In [1]:
%config IPCompleter.greedy=True

In [2]:
import options
import poloniex
import printer
import time, datetime
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
config = options.Options()
polo = poloniex.Trader(config)
P = printer.Printer(config).no_size_printer
P()
msg = "AG TF_STOCKS... AGAIN(part 2) {}".format(datetime.datetime.now())
P(msg)
if False:
    # print some goods
    P("Config")
    print(config)
    P("Poloniex Ticker")
    print(polo())
    P()

|~|  |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |#| |~|
|~|       AG TF_STOCKS... AGAIN(part 2) 2017-11-11 12:35:28.479247       |~|


In [4]:
# RESET the TF defaults
tf.reset_default_graph()
P("Reseting TF graph")
# Start a fresh graph with no configs -- TODO: get some config info for the graph
the_graph = tf.Graph()

|~|                          Reseting TF graph                           |~|


In [5]:
# class to hold helper operations
class subroutine:
    def _create_one_cell(self):
        return tf.contrib.rnn.LSTMCell(config.lstm_size, state_is_tuple=True)
        if config.keep_prob < 1.0:
            return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)
sub = subroutine()

In [6]:
with the_graph.as_default():
    P("Starting to build the_graph")
    # Dimension = (
    #     number of data examples, 
    #     number of input in one computation step, 
    #     number of numbers in one input
    # )
    # We don't know the number of examples beforehand, so it is None.
    inputs = tf.placeholder(tf.float32, [None, config.num_steps, config.input_size], name='AGInputTensor')
    targets = tf.placeholder(tf.float32, [None, config.input_size], name='AGTargetTensor')
    learning_rate = tf.placeholder(tf.float32, None, name='AGLearningRate')
    

|~|                     Starting to build the_graph                      |~|


In [7]:
    cell = tf.contrib.rnn.MultiRNNCell(
        [_create_one_cell() for _ in range(config.num_layers)], 
        state_is_tuple=True
    ) if int(config.num_layers) > 1 else sub._create_one_cell()
    P("Created a cell --> {}".format(cell.name))

|~|                    Created a cell --> lstm_cell_1                    |~|


In [8]:
    val, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    P("Created a dynamic RNN...")


TypeError: A string has ambiguous TensorShape, please wrap in a list or convert to an int: 128